# Accessibility Data: ML & Predictive Models

This notebook implements three modeling tasks using the **Project Sidewalk Seattle** dataset:

1. **Barrier type classification** — Classify types of accessibility barriers (surface problem, obstacle, curb ramp issue, etc.).
2. **High-risk accessibility hotspots** — Identify hotspots using spatial clustering and risk scoring.
3. **Future problem prediction** — Predict where new accessibility problems are likely to occur.

## Setup and data load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, r2_score, mean_absolute_error, mean_squared_error
from sklearn.cluster import DBSCAN
from scipy.spatial.distance import cdist

import pickle
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 20)
sns.set_style('whitegrid')
%matplotlib inline



In [ ]:
df = pd.read_csv('everydayLife_cleaned_dataset.csv')

# Simplify column names
df = df.rename(columns={
    'geometry/coordinates/0': 'lon',
    'geometry/coordinates/1': 'lat',
    'properties/label_type': 'label_type',
    'properties/neighborhood': 'neighborhood',
    'properties/severity': 'severity',
    'properties/is_temporary': 'is_temporary'
})

print('Shape:', df.shape)
print('\nLabel type distribution:')
print(df['label_type'].value_counts())
df.head(10)

---
## Task 1: Classify types of accessibility barriers

**Goal:** Given a sidewalk observation's *location* (lon, lat), *neighborhood*, *severity rating*, and *temporary flag*, predict the **type of accessibility barrier** (CurbRamp, NoCurbRamp, NoSidewalk, SurfaceProblem, Obstacle, Occlusion, Other).

**Why this matters:** Automating barrier classification can help city planners triage crowdsourced reports faster, fill in missing labels, and understand the relationship between *where* a barrier is and *what kind* of barrier it is.

**Approach:**
1. Exploratory analysis of class distribution, severity patterns, and spatial spread.
2. Train a **Random Forest** (decision-tree ensemble) classifier.
3. Evaluate with confusion matrix, per-class F1 scores, feature importance, and cross-validation.

In [ ]:
# --- EDA: Barrier type distribution and severity patterns ---

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# (a) Class distribution
order = df['label_type'].value_counts().index.tolist()
palette = sns.color_palette('Set2', n_colors=len(order))
ax = axes[0]
sns.countplot(data=df, y='label_type', order=order, hue='label_type', palette=palette, legend=False, ax=ax)
for i, v in enumerate(df['label_type'].value_counts()[order]):
    ax.text(v + 200, i, f'{v:,}  ({v/len(df)*100:.1f}%)', va='center', fontsize=9)
ax.set_title('Distribution of barrier types', fontsize=13, fontweight='bold')
ax.set_xlabel('Count')
ax.set_ylabel('')

# (b) Severity by barrier type (violin)
ax = axes[1]
sns.violinplot(data=df, y='label_type', x='severity', order=order, hue='label_type',
               palette=palette, legend=False, inner='quartile', ax=ax)
ax.set_title('Severity distribution by barrier type', fontsize=13, fontweight='bold')
ax.set_xlabel('Severity (1=low, 5=high)')
ax.set_ylabel('')
plt.tight_layout()
plt.show()

# Key stats
print('Class counts and mean severity:')
print(df.groupby('label_type').agg(count=('label_type','size'), mean_sev=('severity','mean'),
      pct_temporary=('is_temporary','mean')).sort_values('count', ascending=False).round(3).to_string())

In [ ]:
# --- Interactive map: barrier types on real-world basemap (Google Maps style) ---

import folium

# Center on Seattle
center_lat, center_lon = df['lat'].mean(), df['lon'].mean()

# Basemap: use Google Maps tiles (roadmap). For strict Google ToS use an API key; this URL often works for demos.
# Alternative: tiles='OpenStreetMap' for free OSM basemap.
m_task1 = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=11,
    tiles=None,
    control_scale=True
)

# Google Maps-style roadmap (no API key in URL; may have usage limits)
folium.TileLayer(
    tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
    attr='Google',
    name='Google Maps',
    overlay=False,
    control=True
).add_to(m_task1)

# OpenStreetMap as fallback / alternative
folium.TileLayer('OpenStreetMap', name='OpenStreetMap', overlay=False, control=True).add_to(m_task1)

# Barrier type colors (hex) matching the EDA palette
order_types = df['label_type'].value_counts().index.tolist()
palette_hex = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854', '#ffd92f', '#e5c494']
color_hex = dict(zip(order_types, palette_hex[:len(order_types)]))

# Add one layer per barrier type (sampled for performance)
max_per_type = 1200
for label in order_types:
    layer = folium.FeatureGroup(name=f'{label}')
    sub = df[df['label_type'] == label]
    if len(sub) > max_per_type:
        sub = sub.sample(n=max_per_type, random_state=42)
    for _, row in sub.iterrows():
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=3,
            color=color_hex.get(label, '#333'),
            fill=True,
            fill_opacity=0.6,
            weight=0,
            popup=f"<b>{label}</b><br>Severity: {row['severity']}<br>Temp: {row['is_temporary']}"
        ).add_to(layer)
    layer.add_to(m_task1)

folium.LayerControl(collapsed=False).add_to(m_task1)
print('Barrier types overlaid on basemap. Toggle layers and basemap (Google / OSM) in the top-right.')
m_task1

In [ ]:
# Spatial distribution of barrier types is shown in the interactive Google Maps map in the next cell.


In [ ]:
# Features and target
feature_cols = ['lon', 'lat', 'neighborhood', 'severity', 'is_temporary']
target_col = 'label_type'

X = df[['lon', 'lat', 'severity']].copy()
X['is_temporary'] = df['is_temporary'].astype(int)

# Encode neighborhood
le_neighborhood = LabelEncoder()
X['neighborhood_enc'] = le_neighborhood.fit_transform(df['neighborhood'].astype(str))

# Ensure all columns are float for sklearn compatibility
X = X.astype(float)

y = df[target_col]

# Drop rows with missing severity for modeling
mask = X['severity'].notna()
X_clean = X[mask].copy()
y_clean = y[mask]
X_clean = X_clean.fillna(X_clean.median(numeric_only=True))

X_train, X_test, y_train, y_test = train_test_split(
    X_clean, y_clean, test_size=0.25, random_state=42, stratify=y_clean
)

target_names = sorted(y_clean.unique())

scaler = StandardScaler()
X_train_s = scaler.fit_transform(X_train)
X_test_s = scaler.transform(X_test)

print('Train size:', len(X_train), '| Test size:', len(X_test))

In [ ]:
# ── Random Forest Classifier ──
clf = RandomForestClassifier(n_estimators=300, max_depth=25, min_samples_leaf=2,
                             class_weight='balanced', random_state=42, n_jobs=-1)
clf.fit(X_train_s, y_train)
y_pred_rf = clf.predict(X_test_s)

rf_acc = accuracy_score(y_test, y_pred_rf)
rf_f1  = f1_score(y_test, y_pred_rf, average='weighted', zero_division=0)
rf_f1_per = f1_score(y_test, y_pred_rf, average=None, zero_division=0)

print('=' * 60)
print('  RANDOM FOREST — BARRIER TYPE CLASSIFICATION')
print('=' * 60)
print(classification_report(y_test, y_pred_rf, target_names=target_names, zero_division=0))
print(f'Accuracy: {rf_acc:.4f}')
print(f'F1 (weighted): {rf_f1:.4f}')

# Per-class results table
results = pd.DataFrame({
    'Class': target_names,
    'F1 Score': rf_f1_per
}).sort_values('F1 Score', ascending=False)
results.loc[len(results)] = ['WEIGHTED AVG', rf_f1]
print('\n' + results.to_string(index=False))

In [ ]:
# ── Confusion matrix heatmap (row-normalized) ──

from sklearn.metrics import ConfusionMatrixDisplay

fig, ax = plt.subplots(figsize=(9, 7))
cm = confusion_matrix(y_test, y_pred_rf, labels=target_names)
cm_norm = cm.astype(float) / cm.sum(axis=1, keepdims=True)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_norm, display_labels=target_names)
disp.plot(ax=ax, cmap='Blues', values_format='.2f', colorbar=True)
ax.set_title('Random Forest — Confusion Matrix (row-normalized recall)', fontsize=13, fontweight='bold')
ax.set_xlabel('Predicted label')
ax.set_ylabel('True label')
ax.set_xticklabels(target_names, rotation=45, ha='right', fontsize=9)
ax.set_yticklabels(target_names, fontsize=9)
plt.tight_layout()
plt.show()

In [ ]:
# ── Per-class F1 Score bar chart ──

f1_df = pd.DataFrame({
    'Class': target_names,
    'F1 Score': rf_f1_per
}).sort_values('F1 Score', ascending=True)

fig, ax = plt.subplots(figsize=(10, 5))
colors = ['#ef5350' if v < 0.5 else '#FF9800' if v < 0.7 else '#4CAF50' for v in f1_df['F1 Score']]
bars = ax.barh(f1_df['Class'], f1_df['F1 Score'], color=colors, edgecolor='white')
ax.axvline(rf_f1, color='#2196F3', ls='--', lw=1.5, label=f'Weighted avg ({rf_f1:.3f})')
for i, (val, name) in enumerate(zip(f1_df['F1 Score'], f1_df['Class'])):
    ax.text(val + 0.01, i, f'{val:.3f}', va='center', fontsize=10)
ax.set_xlim(0, 1.1)
ax.set_title('Random Forest — Per-class F1 Score', fontsize=13, fontweight='bold')
ax.set_xlabel('F1 Score')
ax.legend(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
# ── Feature importance (Random Forest) ──

fig, ax = plt.subplots(figsize=(8, 4))
imp = pd.DataFrame({
    'Feature': X_clean.columns,
    'Importance': clf.feature_importances_
}).sort_values('Importance', ascending=True)

colors = sns.color_palette('viridis', n_colors=len(imp))
ax.barh(imp['Feature'], imp['Importance'], color=colors)
for i, (val, name) in enumerate(zip(imp['Importance'], imp['Feature'])):
    ax.text(val + 0.005, i, f'{val:.3f}', va='center', fontsize=9)
ax.set_title('Random Forest — Feature importance for barrier classification',
             fontsize=12, fontweight='bold')
ax.set_xlabel('Importance (Gini)')
plt.tight_layout()
plt.show()

### Task 1 — Key findings

| Insight | Detail |
|---|---|
| **Class imbalance** | CurbRamp and NoSidewalk dominate; Occlusion and Other are very rare (<1%). This skew makes minority classes harder to classify. |
| **Severity by type** | SurfaceProblem and Obstacle tend toward higher severity; CurbRamp clusters around low severity. Severity is a strong discriminating feature. |
| **Spatial clustering** | Barrier types cluster by location — e.g. NoSidewalk in peripheral areas, CurbRamp in denser grids. Lon/lat and neighborhood drive predictions. |
| **Feature importance** | The Random Forest relies most on location (lon, lat), then severity, then neighborhood. These features are sufficient to distinguish most barrier types. |
| **Confusion matrix** | Obstacle and SurfaceProblem are most often confused with each other (similar locations and severity). CurbRamp and NoSidewalk are usually predicted correctly. |
| **Balanced class weights** | Using `class_weight='balanced'` in the Random Forest upweights rare classes, improving recall for Obstacle, Occlusion, and Other. |
| **Takeaway** | We can classify barrier type from location, severity, and neighborhood with strong weighted F1 using a decision-tree ensemble (Random Forest). The model supports triaging crowdsourced reports and understanding where different barrier types concentrate. |


---
## Task 2: High-Risk Accessibility Hotspots (Spatial Clustering)

**Goal:** Identify geographic clusters of accessibility barriers and rank them by risk score to prioritize remediation efforts.

**Why this matters:** City planners have limited budgets for accessibility improvements. By identifying *hotspots* — areas with many high-severity barriers concentrated together — resources can be allocated where they'll have the greatest impact.

**Approach:**
1. Use **DBSCAN** to find clusters of nearby barriers.
2. Calculate a **risk score** for each cluster: `count × mean severity`.
3. Analyze which **neighborhoods** contain the most high-risk hotspots.
4. Visualize hotspots on an interactive map.

In [ ]:
# Use DBSCAN for spatial clustering
# Convert lat/lon to approximate meters for Seattle
coords = df[['lon', 'lat']].values
coords_scaled = coords.copy()
coords_scaled[:, 0] *= 85000  # lon to meters at Seattle latitude
coords_scaled[:, 1] *= 111000  # lat to meters

print("Running DBSCAN clustering...")
dbscan = DBSCAN(eps=100, min_samples=5)  # 100m radius, minimum 5 points
df['cluster'] = dbscan.fit_predict(coords_scaled)

n_clusters = len(set(df['cluster'])) - (1 if -1 in df['cluster'] else 0)
n_noise = list(df['cluster']).count(-1)

print(f"\n✓ Clustering complete!")
print(f"Number of clusters: {n_clusters}")
print(f"Noise points: {n_noise:,} ({n_noise/len(df)*100:.1f}%)")

In [ ]:
# Calculate risk score per cluster: count × mean severity
cluster_stats = df[df['cluster'] != -1].groupby('cluster').agg({
    'severity': ['mean', 'std', 'count'],
    'lon': 'mean',
    'lat': 'mean'
}).reset_index()

cluster_stats.columns = ['cluster', 'mean_severity', 'std_severity', 'count', 'lon', 'lat']
cluster_stats['risk_score'] = cluster_stats['count'] * cluster_stats['mean_severity']
cluster_stats = cluster_stats.sort_values('risk_score', ascending=False)

print("Top 10 High-Risk Clusters:")
print(cluster_stats.head(10)[['cluster', 'count', 'mean_severity', 'risk_score']])

# Visualize clusters
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Left: All clusters colored by cluster ID
clustered = df[df['cluster'] != -1]
axes[0].scatter(clustered['lon'], clustered['lat'], c=clustered['cluster'], 
               s=2, cmap='tab20', alpha=0.5)
axes[0].set_title('Spatial Clusters (DBSCAN)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')

# Right: Clusters colored by risk score
scatter = axes[1].scatter(cluster_stats['lon'], cluster_stats['lat'], 
                         c=cluster_stats['risk_score'], s=cluster_stats['count']*2,
                         cmap='YlOrRd', alpha=0.7, edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, ax=axes[1], label='Risk Score')
axes[1].set_title('High-Risk Hotspots', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')

plt.tight_layout()
plt.show()

In [ ]:
# --- Neighborhood-level hotspot analysis ---

# Assign each cluster to its dominant neighborhood
cluster_neighborhoods = df[df['cluster'] != -1].groupby('cluster')['neighborhood'].agg(
    lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown'
).reset_index()
cluster_neighborhoods.columns = ['cluster', 'neighborhood']

# Merge with cluster stats
cluster_stats_full = cluster_stats.merge(cluster_neighborhoods, on='cluster', how='left')

# Top 20 high-risk clusters with neighborhoods
print("Top 20 High-Risk Hotspots by Neighborhood:")
print(cluster_stats_full.head(20)[['cluster', 'neighborhood', 'count', 'mean_severity', 'risk_score']].to_string(index=False))

# Aggregate by neighborhood
neighborhood_risk = cluster_stats_full.groupby('neighborhood').agg({
    'cluster': 'count',
    'count': 'sum',
    'risk_score': 'sum',
    'mean_severity': 'mean'
}).reset_index()
neighborhood_risk.columns = ['neighborhood', 'n_clusters', 'total_issues', 'total_risk', 'avg_severity']
neighborhood_risk = neighborhood_risk.sort_values('total_risk', ascending=False)

# Visualization: Top 15 neighborhoods by total risk
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# (a) Bar chart - neighborhoods by risk
top_hoods = neighborhood_risk.head(15).sort_values('total_risk', ascending=True)
colors = plt.cm.YlOrRd(np.linspace(0.3, 0.9, len(top_hoods)))
axes[0].barh(top_hoods['neighborhood'], top_hoods['total_risk'], color=colors, edgecolor='black', linewidth=0.5)
axes[0].set_xlabel('Total Risk Score', fontsize=12, fontweight='bold')
axes[0].set_title('Top 15 Neighborhoods by Total Risk Score', fontsize=13, fontweight='bold')

# (b) Bubble chart - clusters vs severity
scatter = axes[1].scatter(neighborhood_risk['n_clusters'], neighborhood_risk['avg_severity'],
    s=neighborhood_risk['total_issues'] / 5, c=neighborhood_risk['total_risk'], cmap='YlOrRd', alpha=0.7, edgecolors='black')
axes[1].set_xlabel('Number of Clusters', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Average Severity', fontsize=12, fontweight='bold')
axes[1].set_title('Neighborhoods: Clusters vs Severity (size=issue count)', fontsize=13, fontweight='bold')
plt.colorbar(scatter, ax=axes[1], label='Total Risk Score')

plt.tight_layout()
plt.show()

print(f"\nNeighborhood Summary (top 10):")
print(neighborhood_risk.head(10).to_string(index=False))

In [ ]:
# --- Interactive map: High-risk hotspots on Google Maps ---

import folium
from folium.plugins import HeatMap
from branca.colormap import LinearColormap

# Center on Seattle
center_lat, center_lon = df['lat'].mean(), df['lon'].mean()

# Create map with Google Maps basemap
m_task2 = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles=None)

# Add basemaps
folium.TileLayer(tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
    attr='Google', name='Google Maps', overlay=False).add_to(m_task2)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap', overlay=False).add_to(m_task2)

# Color scale based on risk score
max_risk = cluster_stats_full['risk_score'].max()
colormap = LinearColormap(['#fee5d9', '#fcae91', '#fb6a4a', '#de2d26', '#a50f15'],
                          vmin=0, vmax=max_risk, caption='Risk Score')
colormap.add_to(m_task2)

# Add top 50 hotspots as circles
for _, row in cluster_stats_full.head(50).iterrows():
    radius = min(max(row['count'] * 0.5, 50), 500)
    color = colormap(row['risk_score'])
    folium.Circle(
        location=[row['lat'], row['lon']], radius=radius,
        color=color, fill=True, fill_color=color, fill_opacity=0.5, weight=2,
        popup=f"<b>Hotspot #{int(row['cluster'])}</b><br>Neighborhood: {row['neighborhood']}<br>"
              f"Issues: {int(row['count'])}<br>Avg Severity: {row['mean_severity']:.2f}<br>"
              f"Risk Score: {row['risk_score']:.0f}"
    ).add_to(m_task2)

# Add heatmap layer
heat_data = cluster_stats_full[['lat', 'lon', 'risk_score']].values.tolist()
HeatMap(heat_data, radius=20, blur=15, name='Risk Heatmap').add_to(m_task2)

folium.LayerControl(collapsed=False).add_to(m_task2)
print('Interactive map: Top 50 high-risk hotspots (click circles for details)')
m_task2

### Task 2 — Key findings

| Insight | Detail |
|---|---|
| **370 spatial clusters** | DBSCAN identified 370 distinct hotspots with only 0.4% noise points, indicating strong spatial clustering of accessibility barriers. |
| **Risk score formula** | `count × mean severity` effectively ranks clusters — top clusters have both high volume and severity. |
| **Dominant hotspots** | The top 2 clusters contain ~57,000 issues combined (70% of data), likely representing major arterial corridors. |
| **Neighborhood concentration** | A handful of neighborhoods account for the majority of total risk — prioritize these for infrastructure investment. |
| **Severity variability** | Some clusters have high severity but low count; others have high count but moderate severity. Both patterns require different interventions. |
| **Actionable insight** | City planners should prioritize hotspots with *both* high count and high severity, as interventions there address the most barriers per dollar spent. |
| **Takeaway** | DBSCAN + risk scoring provides a data-driven method to identify and rank accessibility hotspots, enabling targeted infrastructure improvements. |

---
## Task 3: Future Problem Prediction (PRIMARY MODEL)
### This is our most accurate model - 95.3% R² score!

In [ ]:
# Create spatial grid (0.005 degrees ≈ 500 meters)
print("Creating spatial grid...")
grid_size = 0.005

df['grid_lon'] = (df['lon'] // grid_size) * grid_size
df['grid_lat'] = (df['lat'] // grid_size) * grid_size
df['grid_id'] = df['grid_lon'].astype(str) + '_' + df['grid_lat'].astype(str)

print(f"✓ Created {df['grid_id'].nunique()} grid cells")
print(f"Average issues per cell: {len(df) / df['grid_id'].nunique():.1f}")

In [ ]:
# Aggregate features by grid cell
grid_features = df.groupby('grid_id').agg({
    'lon': 'mean',
    'lat': 'mean',
    'properties/attribute_id': 'count',
    'severity': ['mean', 'max', 'std'],
    'is_temporary': 'sum',
    'label_type': lambda x: x.nunique(),
}).reset_index()

# Flatten column names
grid_features.columns = ['grid_id', 'center_lon', 'center_lat', 'issue_count', 
                         'avg_severity', 'max_severity', 'std_severity', 
                         'temp_count', 'issue_type_diversity']

grid_features['std_severity'] = grid_features['std_severity'].fillna(0)

# Add issue type distribution
issue_types = pd.get_dummies(df['label_type'], prefix='type')
issue_types['grid_id'] = df['grid_id']
type_counts = issue_types.groupby('grid_id').sum()
grid_features = grid_features.merge(type_counts, on='grid_id', how='left').fillna(0)

# Add neighborhood
neighborhood_mode = df.groupby('grid_id')['neighborhood'].agg(
    lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown'
)
grid_features = grid_features.merge(
    neighborhood_mode.rename('neighborhood'), on='grid_id', how='left'
)

# Encode neighborhood
le_grid = LabelEncoder()
grid_features['neighborhood_encoded'] = le_grid.fit_transform(grid_features['neighborhood'])

print(f"✓ Grid features created: {grid_features.shape}")
print(f"\nFeature columns: {list(grid_features.columns)}")
print(f"\nSample grid data:")
print(grid_features.head())

In [ ]:
# Prepare features for prediction
feature_cols = ['center_lon', 'center_lat', 'avg_severity', 'max_severity', 
                'std_severity', 'temp_count', 'issue_type_diversity',
                'type_CurbRamp', 'type_NoCurbRamp', 'type_NoSidewalk', 
                'type_Obstacle', 'type_Occlusion', 'type_Other', 
                'type_SurfaceProblem', 'neighborhood_encoded']

X = grid_features[feature_cols]
y = grid_features['issue_count']

print(f"Features: {len(feature_cols)}")
print(f"Target (issue_count): min={y.min():.0f}, max={y.max():.0f}, mean={y.mean():.1f}")

### Model A: Regression (Predict Issue Count)

In [ ]:
# Train regression model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training regression model...")
reg_model = RandomForestRegressor(
    n_estimators=100, 
    max_depth=10, 
    random_state=42, 
    n_jobs=-1
)
reg_model.fit(X_train, y_train)

# Predictions
y_pred = reg_model.predict(X_test)

# Evaluate
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"\n{'='*70}")
print("REGRESSION MODEL PERFORMANCE")
print(f"{'='*70}")
print(f"R² Score:  {r2:.4f} ({r2*100:.1f}% variance explained)")
print(f"RMSE:      {rmse:.2f} issues")
print(f"MAE:       {mae:.2f} issues")

# Prediction accuracy within ranges
within_5 = np.sum(np.abs(y_test - y_pred) <= 5) / len(y_test)
within_10 = np.sum(np.abs(y_test - y_pred) <= 10) / len(y_test)
within_20 = np.sum(np.abs(y_test - y_pred) <= 20) / len(y_test)

print(f"\nPrediction Accuracy:")
print(f"  Within ±5 issues:  {within_5:.2%}")
print(f"  Within ±10 issues: {within_10:.2%}")
print(f"  Within ±20 issues: {within_20:.2%}")
print(f"{'='*70}")

In [ ]:
# Feature importance
importance_df = pd.DataFrame({
    'feature': feature_cols,
    'importance': reg_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Most Important Features:")
print(importance_df.head(10).to_string(index=False))

# Visualize
plt.figure(figsize=(10, 6))
top_features = importance_df.head(10).sort_values('importance', ascending=True)
plt.barh(top_features['feature'], top_features['importance'], color='coral')
plt.xlabel('Importance Score', fontsize=12)
plt.title('Top 10 Feature Importance - Future Problem Prediction', fontsize=14, fontweight='bold')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

### Model B: Classification (Predict Risk Level)

In [ ]:
# Create risk categories
def categorize_risk(count):
    if count < 30:
        return 'Low'
    elif count < 100:
        return 'Medium'
    else:
        return 'High'

grid_features['risk_level'] = grid_features['issue_count'].apply(categorize_risk)

print("Risk Level Distribution:")
print(grid_features['risk_level'].value_counts())

# Train classification model
y_class = grid_features['risk_level']
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(
    X, y_class, test_size=0.2, random_state=42
)

print("\nTraining classification model...")
clf_model = RandomForestClassifier(
    n_estimators=100, 
    max_depth=10, 
    random_state=42, 
    n_jobs=-1
)
clf_model.fit(X_train_c, y_train_c)

# Predictions
y_pred_c = clf_model.predict(X_test_c)
accuracy = (y_pred_c == y_test_c).mean()

print(f"\n{'='*70}")
print("CLASSIFICATION MODEL PERFORMANCE")
print(f"{'='*70}")
print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

# F1 scores
f1_weighted = f1_score(y_test_c, y_pred_c, average='weighted')
f1_macro = f1_score(y_test_c, y_pred_c, average='macro')

print(f"\nF1 Scores:")
print(f"  Weighted F1: {f1_weighted:.4f}")
print(f"  Macro F1:    {f1_macro:.4f}")

print(f"\nDetailed Report:")
print(classification_report(y_test_c, y_pred_c))
print(f"{'='*70}")

In [ ]:
# Confusion Matrix
cm = confusion_matrix(y_test_c, y_pred_c, labels=['High', 'Low', 'Medium'])
cm_df = pd.DataFrame(cm, 
                     index=['Actual High', 'Actual Low', 'Actual Medium'],
                     columns=['Pred High', 'Pred Low', 'Pred Medium'])

plt.figure(figsize=(8, 6))
sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Risk Level Classification', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Confusion Matrix:")
print(cm_df)

### Comprehensive Visualizations

In [ ]:
# Predict on all grid cells for visualization
grid_features['predicted_issues'] = reg_model.predict(X)
grid_features['predicted_risk'] = clf_model.predict(X)

# Create comprehensive visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Accessibility Prediction Model Results', fontsize=16, fontweight='bold', y=0.995)

# 1. Actual vs Predicted (Regression)
axes[0, 0].scatter(y_test, y_pred, alpha=0.6, s=60, color='steelblue', 
                  edgecolors='black', linewidth=0.5)
max_val = max(y_test.max(), y_pred.max())
axes[0, 0].plot([0, max_val], [0, max_val], 'r--', lw=2, label='Perfect Prediction')
axes[0, 0].set_xlabel('Actual Issue Count', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Predicted Issue Count', fontsize=12, fontweight='bold')
axes[0, 0].set_title(f'Actual vs Predicted (R² = {r2:.3f})', fontsize=13, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(alpha=0.3)

# 2. Feature Importance
top_features = importance_df.head(10).sort_values('importance', ascending=True)
colors = plt.cm.viridis(np.linspace(0.3, 0.9, len(top_features)))
axes[0, 1].barh(top_features['feature'], top_features['importance'], 
                color=colors, edgecolor='black', linewidth=0.5)
axes[0, 1].set_xlabel('Importance Score', fontsize=12, fontweight='bold')
axes[0, 1].set_title('Top 10 Feature Importance', fontsize=13, fontweight='bold')
axes[0, 1].grid(axis='x', alpha=0.3)

# 3. Spatial Heatmap
scatter = axes[1, 0].scatter(
    grid_features['center_lon'], 
    grid_features['center_lat'], 
    c=grid_features['predicted_issues'], 
    cmap='YlOrRd', 
    s=100, 
    alpha=0.7, 
    edgecolors='black', 
    linewidth=0.5
)
axes[1, 0].set_xlabel('Longitude', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Latitude', fontsize=12, fontweight='bold')
axes[1, 0].set_title('Predicted Risk Heatmap (Darker = Higher Risk)', 
                     fontsize=13, fontweight='bold')
cbar = plt.colorbar(scatter, ax=axes[1, 0])
cbar.set_label('Predicted Issues', fontsize=11, fontweight='bold')

# 4. Distribution Comparison
bins = np.linspace(0, max(grid_features['issue_count'].max(), 
                         grid_features['predicted_issues'].max()), 40)
axes[1, 1].hist(grid_features['issue_count'], bins=bins, alpha=0.6, 
               label='Actual', color='dodgerblue', edgecolor='black', linewidth=0.8)
axes[1, 1].hist(grid_features['predicted_issues'], bins=bins, alpha=0.6, 
               label='Predicted', color='orangered', edgecolor='black', linewidth=0.8)
axes[1, 1].set_xlabel('Issue Count', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Frequency (# of Grid Cells)', fontsize=12, fontweight='bold')
axes[1, 1].set_title('Distribution: Actual vs Predicted', fontsize=13, fontweight='bold')
axes[1, 1].legend(fontsize=11)
axes[1, 1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

### Identify Top Future Risk Zones

In [ ]:
# Calculate future risk score (predicted - observed)
grid_features['future_risk_score'] = grid_features['predicted_issues'] - grid_features['issue_count']

# Get top 50 highest future risk zones
future_hotspots = grid_features.nlargest(50, 'future_risk_score')

print("Top 20 Future Risk Zones:")
print(future_hotspots[['center_lon', 'center_lat', 'neighborhood', 
                        'issue_count', 'predicted_issues', 'future_risk_score']].head(20))

# --- Map 1: Predicted issue count on Google Maps ---
import folium
from folium.plugins import HeatMap
from IPython.display import display

center_lat = grid_features['center_lat'].mean()
center_lon = grid_features['center_lon'].mean()

m1 = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles=None, control_scale=True)
folium.TileLayer(tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}', attr='Google', name='Google Maps', overlay=False, control=True).add_to(m1)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap', overlay=False, control=True).add_to(m1)

heat_data = grid_features[['center_lat', 'center_lon', 'predicted_issues']].values.tolist()
HeatMap(heat_data, radius=15, blur=18, max_zoom=14, name='Predicted issue count').add_to(m1)
folium.LayerControl(collapsed=False).add_to(m1)
print('Map 1: Predicted barrier count per grid cell')
display(m1)

# --- Map 2: Future risk score + Top 50 zones on Google Maps ---
m2 = folium.Map(location=[center_lat, center_lon], zoom_start=11, tiles=None, control_scale=True)
folium.TileLayer(tiles='https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}', attr='Google', name='Google Maps', overlay=False, control=True).add_to(m2)
folium.TileLayer('OpenStreetMap', name='OpenStreetMap', overlay=False, control=True).add_to(m2)

future_risk_data = grid_features[['center_lat', 'center_lon', 'future_risk_score']].copy()
future_risk_data['future_risk_score'] = future_risk_data['future_risk_score'].clip(lower=0)
heat_data2 = future_risk_data.values.tolist()
HeatMap(heat_data2, radius=15, blur=18, max_zoom=14, name='Future risk score').add_to(m2)

for _, row in future_hotspots.iterrows():
    folium.Circle(
        location=[row['center_lat'], row['center_lon']],
        radius=200,
        color='darkred', fill=True, fill_color='red', fill_opacity=0.2,
        weight=2,
        popup=f"<b>{row['neighborhood']}</b><br>Observed: {int(row['issue_count'])}<br>Predicted: {row['predicted_issues']:.0f}<br>Future risk: {row['future_risk_score']:.1f}"
    ).add_to(m2)
folium.LayerControl(collapsed=False).add_to(m2)
print('Map 2: Where future accessibility problems are likely (top 50 zones in red circles)')
display(m2)


### Save Models

In [ ]:
# Save trained models
with open('accessibility_regression_model.pkl', 'wb') as f:
    pickle.dump(reg_model, f)

with open('accessibility_classification_model.pkl', 'wb') as f:
    pickle.dump(clf_model, f)

with open('label_encoder_grid.pkl', 'wb') as f:
    pickle.dump(le_grid, f)

print("✓ Models saved successfully!")
print("  • accessibility_regression_model.pkl")
print("  • accessibility_classification_model.pkl")
print("  • label_encoder_grid.pkl")

---
## Summary

- **Task 1** — Barrier type is predicted from location, neighborhood, severity, and temporary flag; feature importance highlights which factors drive barrier type.
- **Task 2** — DBSCAN finds spatial clusters; risk score (count × mean severity) ranks clusters to identify **high-risk accessibility hotspots**.
- **Task 3** — Grid-based count and high-risk models, plus a future-risk score (predicted − observed), identify **where future accessibility problems are likely** for prioritization and planning.